In [6]:
from google.colab import files
uploaded = files.upload()


Saving WA_Fn-UseC_-HR-Employee-Attrition.csv to WA_Fn-UseC_-HR-Employee-Attrition.csv


In [7]:
import os

print(os.listdir())


['.config', 'WA_Fn-UseC_-HR-Employee-Attrition.csv', 'sample_data']


In [8]:
import pandas as pd

df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [9]:
print(df.shape); print(df.columns.tolist())
print(df['Attrition'].value_counts())
print(df.isna().sum().sort_values(ascending=False).head())
df.head()


(1470, 35)
['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
Attrition
No     1233
Yes     237
Name: count, dtype: int64
Age               0
Attrition         0
BusinessTravel    0
DailyRate         0
Department        0
dtype: int64


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [10]:
df['AttritionFlag'] = df['Attrition'].map({'Yes':1,'No':0}).astype(int)


In [11]:
features = ['Age','MonthlyIncome','JobRole','TotalWorkingYears','WorkLifeBalance','DistanceFromHome','YearsSinceLastPromotion','PerformanceRating','OverTime']
X = df[features].copy(); y = df['AttritionFlag']


In [13]:
X['OverTime'] = X['OverTime'].astype(str)



In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_features = [
    'Age','MonthlyIncome','TotalWorkingYears',
    'DistanceFromHome','YearsSinceLastPromotion','PerformanceRating'
]

categorical_features = ['JobRole','WorkLifeBalance','OverTime']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
])


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

pipe_lr = Pipeline([('pre', preprocessor), ('clf', LogisticRegression(class_weight='balanced', solver='liblinear'))])
pipe_rf = Pipeline([('pre', preprocessor), ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))])
pipe_svc = Pipeline([('pre', preprocessor), ('clf', SVC(probability=True, class_weight='balanced'))])


In [19]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
cv = StratifiedKFold(5, shuffle=True, random_state=42)
gs_lr = GridSearchCV(pipe_lr, {'clf__C':[0.1,1,10]}, cv=cv, scoring='roc_auc', n_jobs=-1)
gs_rf = GridSearchCV(pipe_rf, {'clf__n_estimators':[100,200]}, cv=cv, scoring='roc_auc', n_jobs=-1)
gs_svc = GridSearchCV(pipe_svc, {'clf__C':[0.1,1],'clf__kernel':['rbf','linear']}, cv=cv, scoring='roc_auc', n_jobs=-1)
gs_lr.fit(X_train, y_train); gs_rf.fit(X_train, y_train); gs_svc.fit(X_train, y_train)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'MonthlyIncome',
                                                                          'TotalWorkingYears',
                                                                          'DistanceFromHome',
                                                                          'YearsSinceLastPromotion',
                                                                          'PerformanceRating']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['JobRole',
                                                                          'WorkLifeBalance',
                                                                          'OverTime'])])),
                                       ('clf',
                                        SVC(class_weight='balanced',
                                            probability=True))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.1, 1], 'clf__kernel': ['rbf', 'linear']},
             scoring='roc_auc')

In [20]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
for name, gs in [('LR', gs_lr), ('RF', gs_rf), ('SVC', gs_svc)]:
    y_proba = gs.predict_proba(X_test)[:,1]
    y_pred = gs.predict(X_test)
    print(name, 'ROC AUC:', roc_auc_score(y_test, y_proba))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


LR ROC AUC: 0.7125506072874493
              precision    recall  f1-score   support

           0       0.91      0.73      0.81       247
           1       0.31      0.64      0.42        47

    accuracy                           0.72       294
   macro avg       0.61      0.69      0.62       294
weighted avg       0.82      0.72      0.75       294

[[181  66]
 [ 17  30]]
RF ROC AUC: 0.7417951589284176
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       247
           1       0.48      0.21      0.29        47

    accuracy                           0.84       294
   macro avg       0.67      0.58      0.60       294
weighted avg       0.80      0.84      0.81       294

[[236  11]
 [ 37  10]]
SVC ROC AUC: 0.7277112585063313
              precision    recall  f1-score   support

           0       0.92      0.74      0.82       247
           1       0.33      0.66      0.44        47

    accuracy                           0.7

In [21]:
from sklearn.ensemble import VotingClassifier
best_lr = gs_lr.best_estimator_; best_rf = gs_rf.best_estimator_; best_svc = gs_svc.best_estimator_
ensemble = VotingClassifier(
    estimators=[('lr', best_lr), ('rf', best_rf), ('svc', best_svc)],
    voting='soft',
    weights=[1, 3, 2]
)
ensemble.fit(X_train, y_train)


VotingClassifier(estimators=[('lr',
                              Pipeline(steps=[('pre',
                                               ColumnTransformer(transformers=[('num',
                                                                                StandardScaler(),
                                                                                ['Age',
                                                                                 'MonthlyIncome',
                                                                                 'TotalWorkingYears',
                                                                                 'DistanceFromHome',
                                                                                 'YearsSinceLastPromotion',
                                                                                 'PerformanceRating']),
                                                                               ('cat',
                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                              sparse_output=False),
                                                                                ['JobRole',
                                                                                 'WorkLifeBalance',
                                                                                 'OverTime'])])),
                                              ('clf',
                                               Logistic...
                                               ColumnTransformer(transformers=[('num',
                                                                                StandardScaler(),
                                                                                ['Age',
                                                                                 'MonthlyIncome',
                                                                                 'TotalWorkingYears',
                                                                                 'DistanceFromHome',
                                                                                 'YearsSinceLastPromotion',
                                                                                 'PerformanceRating']),
                                                                               ('cat',
                                                                                OneHotEncoder(handle_unknown='ignore',
                                                                                              sparse_output=False),
                                                                                ['JobRole',
                                                                                 'WorkLifeBalance',
                                                                                 'OverTime'])])),
                                              ('clf',
                                               SVC(C=1, class_weight='balanced',
                                                   kernel='linear',
                                                   probability=True))]))],
                 voting='soft', weights=[1, 3, 2])

In [22]:
y_proba = ensemble.predict_proba(X_test)[:,1]
y_pred = ensemble.predict(X_test)
print("Ensemble ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

from sklearn.metrics import precision_recall_curve
prec, rec, thr = precision_recall_curve(y_test, y_proba)


Ensemble ROC AUC: 0.7352915841157721
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       247
           1       0.47      0.19      0.27        47

    accuracy                           0.84       294
   macro avg       0.67      0.58      0.59       294
weighted avg       0.80      0.84      0.81       294



In [24]:
rec_target = 0.80
candidates = [(t,p,r) for p,r,t in zip(prec, rec, thr) if r >= rec_target]
if candidates:
    print("Example threshold achieving recall >= {:.2f}: precision {:.3f}, threshold {:.3f}".format(rec_target, candidates[-1][0], candidates[-1][2]))
else:
    print(f"No threshold produced recall >= {rec_target} on test set (consider resampling or lower target).")


Example threshold achieving recall >= 0.80: precision 0.133, threshold 0.809


In [25]:
pre = best_rf.named_steps['pre']
rf_est = best_rf.named_steps['clf']

feature_names = []
for name, transformer, cols in pre.transformers_:
    if transformer == 'drop':
        continue
    if transformer == 'passthrough':

        if isinstance(cols, (list, tuple)):
            feature_names.extend(list(cols))
        continue

    if hasattr(transformer, 'named_steps'):
        last_step = list(transformer.named_steps.items())[-1][1]
        actual = last_step
    else:
        actual = transformer

    if actual.__class__.__name__ == 'OneHotEncoder':
        try:
            names = actual.get_feature_names_out(cols)
        except Exception:
            names = actual.get_feature_names(cols)
        feature_names.extend(list(names))
    else:
        if isinstance(cols, (list, tuple)):
            feature_names.extend(list(cols))
        else:
            feature_names.append(str(cols))

n_model = len(rf_est.feature_importances_)
n_names = len(feature_names)
print(f"\nRF importances length: {n_model}, extracted feature names length: {n_names}")

if n_model == n_names:
    imp_df = pd.DataFrame({'feature': feature_names, 'importance': rf_est.feature_importances_})
    imp_df = imp_df.sort_values('importance', ascending=False).reset_index(drop=True)
    print("\nTop RF features:\n", imp_df.head(15).to_string(index=False))
else:
    print("WARNING: mismatch between RF importances and extracted names.")
    print("Transformed shape debug:")
    try:
        sample = X_train.iloc[:5]
        transformed = pre.transform(sample)
        print("pre.transform(X_sample).shape:", transformed.shape)
    except Exception as e:
        print("Could not transform sample for debug:", e)

    fallback = pd.Series(rf_est.feature_importances_).sort_values(ascending=False).head(20)
    print("\nTop RF importances by index (fallback):\n", fallback.to_string())


RF importances length: 21, extracted feature names length: 21

Top RF features:
                       feature  importance
                MonthlyIncome    0.198582
                          Age    0.151990
            TotalWorkingYears    0.142501
             DistanceFromHome    0.128863
      YearsSinceLastPromotion    0.076205
                 OverTime_Yes    0.041337
                  OverTime_No    0.035048
      JobRole_Sales Executive    0.025724
            WorkLifeBalance_3    0.024513
   JobRole_Research Scientist    0.023864
JobRole_Laboratory Technician    0.022763
            WorkLifeBalance_2    0.018608
            WorkLifeBalance_1    0.018322
            PerformanceRating    0.017544
 JobRole_Sales Representative    0.015505


In [32]:

try:
    import shap
    shap_available = True
except Exception:
    shap_available = False


In [34]:
RANDOM_STATE = 42
if shap_available and n_model == n_names:
    try:
        print("\nComputing SHAP values for RandomForest (may take a moment)...")
        # Use a small sample to speed up
        X_sample = X_test.sample(min(200, len(X_test)), random_state=RANDOM_STATE)
        X_sample_trans = pre.transform(X_sample)
        explainer = shap.TreeExplainer(rf_est)
        shap_values = explainer.shap_values(X_sample_trans)
        # show one example
        i = 0
        # Adjust indexing for shap_values to correctly extract feature contributions
        s = pd.Series(shap_values[:, i, 1], index=feature_names).sort_values(ascending=False)
        print("\nExample SHAP top positive contributors (push to attrition):\n", s.head(8).to_string())
        print("\nExample SHAP top negative contributors (push to stay):\n", s.tail(8).to_string())
    except Exception as e:
        print("SHAP failed:", e)
else:
    if not shap_available:
        print("\nSHAP not installed — install with `pip install shap` to get per-instance explanations.")
    else:
        print("\nSkipping SHAP because feature-name length mismatch.")


Computing SHAP values for RandomForest (may take a moment)...
SHAP failed: Length of values (200) does not match length of index (21)


In [36]:

import os
import joblib
import warnings
warnings.filterwarnings("ignore")

In [38]:
joblib.dump(ensemble, "BEST_Attrition_Ensemble.joblib")

['BEST_Attrition_Ensemble.joblib']

In [39]:
loaded = joblib.load("BEST_Attrition_Ensemble.joblib")
print("Loaded model:", type(loaded))

Loaded model: <class 'sklearn.ensemble._voting.VotingClassifier'>


In [41]:
import numpy as np

In [42]:
def sample_random_rows(base_df, n=3, seed=RANDOM_STATE):
    rng = np.random.RandomState(seed)
    rows = []
    for _ in range(n):
        rows.append({
            'Age': int(rng.choice(base_df['Age'].values)),
            'MonthlyIncome': int(rng.choice(base_df['MonthlyIncome'].values)),
            'JobRole': str(rng.choice(base_df['JobRole'].values)),
            'TotalWorkingYears': int(rng.choice(base_df['TotalWorkingYears'].values)),
            'WorkLifeBalance': int(rng.choice(base_df['WorkLifeBalance'].values)),
            'DistanceFromHome': int(rng.choice(base_df['DistanceFromHome'].values)),
            'YearsSinceLastPromotion': int(rng.choice(base_df['YearsSinceLastPromotion'].values)),
            'PerformanceRating': int(rng.choice(base_df['PerformanceRating'].values)),
            'OverTime': str(rng.choice(base_df['OverTime'].values))
        })
    return pd.DataFrame(rows)

new_rows = sample_random_rows(df, n=4)
probs_new = loaded.predict_proba(new_rows)[:,1]
preds_new = (probs_new >= 0.5).astype(int)
new_rows['attrition_prob'] = probs_new
new_rows['attrition_pred'] = preds_new

print("\nCustom/random predictions:\n", new_rows)


Custom/random predictions:
    Age  MonthlyIncome                JobRole  TotalWorkingYears  \
0   50           4025     Research Scientist                 11   
1   23           5257  Laboratory Technician                 15   
2   29          19431        Sales Executive                  3   
3   28           3407     Research Scientist                 19   

   WorkLifeBalance  DistanceFromHome  YearsSinceLastPromotion  \
0                2                28                        1   
1                3                17                        1   
2                4                 2                        6   
3                2                23                        1   

   PerformanceRating OverTime  attrition_prob  attrition_pred  
0                  3       No        0.134382               0  
1                  4       No        0.240804               0  
2                  4      Yes        0.436155               0  
3                  3       No        0.131584        

In [43]:
def suggestions_for_row(row, df_ref=df):
    suggestions = []
    if str(row['OverTime']).lower().startswith('y'):
        suggestions.append("Reduce overtime / offer flexible hours")
    if int(row['MonthlyIncome']) < df_ref['MonthlyIncome'].quantile(0.25):
        suggestions.append("Consider salary review / retention bonus")
    try:
        if int(row['WorkLifeBalance']) <= 2:
            suggestions.append("Improve work-life balance (remote days)")
    except:
        pass
    if int(row['YearsSinceLastPromotion']) >= 4:
        suggestions.append("Career path / promotion / training")
    if int(row['DistanceFromHome']) >= 20:
        suggestions.append("Remote option / relocation support")
    if not suggestions:
        suggestions.append("Manager check-in and engagement survey")
    return "; ".join(suggestions)

new_rows['suggestions'] = new_rows.apply(lambda r: suggestions_for_row(r, df), axis=1)
print("\nPredictions + suggestions:\n", new_rows[['attrition_prob','attrition_pred','suggestions']])


Predictions + suggestions:
    attrition_prob  attrition_pred  \
0        0.134382               0   
1        0.240804               0   
2        0.436155               0   
3        0.131584               0   

                                         suggestions  
0  Improve work-life balance (remote days); Remot...  
1             Manager check-in and engagement survey  
2  Reduce overtime / offer flexible hours; Career...  
3  Improve work-life balance (remote days); Remot...  


In [44]:

manual = pd.DataFrame([{
    'Age': 28, 'MonthlyIncome': 3000, 'JobRole': 'Sales Executive',
    'TotalWorkingYears': 3, 'WorkLifeBalance': 2, 'DistanceFromHome': 25,
    'YearsSinceLastPromotion': 6, 'PerformanceRating': 3, 'OverTime': 'Yes'
}])

prob = loaded.predict_proba(manual)[:,1][0]
label = "Yes" if prob >= 0.5 else "No"
print("Manual input prediction:")
print(manual.to_dict(orient='records')[0])
print("Attrition probability:", round(prob,3))
print("Predicted attrition:", label)
print("Suggestions:", suggestions_for_row(manual.iloc[0], df))


Manual input prediction:
{'Age': 28, 'MonthlyIncome': 3000, 'JobRole': 'Sales Executive', 'TotalWorkingYears': 3, 'WorkLifeBalance': 2, 'DistanceFromHome': 25, 'YearsSinceLastPromotion': 6, 'PerformanceRating': 3, 'OverTime': 'Yes'}
Attrition probability: 0.523
Predicted attrition: Yes
Suggestions: Reduce overtime / offer flexible hours; Improve work-life balance (remote days); Career path / promotion / training; Remote option / relocation support


In [45]:
manual = pd.DataFrame([{
    'Age': 38,
    'MonthlyIncome': 9000,
    'JobRole': 'Research Scientist',
    'TotalWorkingYears': 12,
    'WorkLifeBalance': 4,
    'DistanceFromHome': 5,
    'YearsSinceLastPromotion': 1,
    'PerformanceRating': 3,
    'OverTime': 'No'
}])

prob = loaded.predict_proba(manual)[:,1][0]
label = "Yes" if prob >= 0.5 else "No"
print("Manual input prediction:")
print(manual.to_dict(orient='records')[0])
print("Attrition probability:", round(prob,3))
print("Predicted attrition:", label)
print("Suggestions:", suggestions_for_row(manual.iloc[0], df))

Manual input prediction:
{'Age': 38, 'MonthlyIncome': 9000, 'JobRole': 'Research Scientist', 'TotalWorkingYears': 12, 'WorkLifeBalance': 4, 'DistanceFromHome': 5, 'YearsSinceLastPromotion': 1, 'PerformanceRating': 3, 'OverTime': 'No'}
Attrition probability: 0.103
Predicted attrition: No
Suggestions: Manager check-in and engagement survey


In [47]:
manual = pd.DataFrame([{
    'Age': 29,
    'MonthlyIncome': 7200,
    'JobRole': 'Human Resources',
    'TotalWorkingYears': 6,
    'WorkLifeBalance': 4,
    'DistanceFromHome': 3,
    'YearsSinceLastPromotion': 0,
    'PerformanceRating': 3,
    'OverTime': 'No'
}])

prob = loaded.predict_proba(manual)[:,1][0]
label = "Yes" if prob >= 0.5 else "No"
print("Manual input prediction:")
print(manual.to_dict(orient='records')[0])
print("Attrition probability:", round(prob,3))
print("Predicted attrition:", label)
print("Suggestions:", suggestions_for_row(manual.iloc[0], df))

Manual input prediction:
{'Age': 29, 'MonthlyIncome': 7200, 'JobRole': 'Human Resources', 'TotalWorkingYears': 6, 'WorkLifeBalance': 4, 'DistanceFromHome': 3, 'YearsSinceLastPromotion': 0, 'PerformanceRating': 3, 'OverTime': 'No'}
Attrition probability: 0.218
Predicted attrition: No
Suggestions: Manager check-in and engagement survey


In [48]:
manual = pd.DataFrame([{
    'Age': 27,
    'MonthlyIncome': 3200,
    'JobRole': 'Sales Executive',
    'TotalWorkingYears': 3,
    'WorkLifeBalance': 1,
    'DistanceFromHome': 22,
    'YearsSinceLastPromotion': 3,
    'PerformanceRating': 3,
    'OverTime': 'Yes'
}])
prob = loaded.predict_proba(manual)[:,1][0]
label = "Yes" if prob >= 0.5 else "No"
print("Manual input prediction:")
print(manual.to_dict(orient='records')[0])
print("Attrition probability:", round(prob,3))
print("Predicted attrition:", label)
print("Suggestions:", suggestions_for_row(manual.iloc[0], df))

Manual input prediction:
{'Age': 27, 'MonthlyIncome': 3200, 'JobRole': 'Sales Executive', 'TotalWorkingYears': 3, 'WorkLifeBalance': 1, 'DistanceFromHome': 22, 'YearsSinceLastPromotion': 3, 'PerformanceRating': 3, 'OverTime': 'Yes'}
Attrition probability: 0.603
Predicted attrition: Yes
Suggestions: Reduce overtime / offer flexible hours; Improve work-life balance (remote days); Remote option / relocation support
